In [3]:
import numpy as np
import pandas as pd
import re as re
import xlsxwriter as xlsx
import xlrd as xlrd
import csv
from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

sheet = "Base calcul Volumes"
data_file = "C:\\Users\\T0230575\\Desktop\\transfered session A\\PycharmProjects\\TransposeWidetoLong\\data\\2020 11 04 Base calcul volume d'engagements LAS GTI consolidé AZ.xlsx"

cols_to_skip = [0, 1]
yr_toexclude = ['2022','2023']
gbu_toexclude = ['GLOBAL','AVS','GTS','DMS','SIX','LAS']
seg_toexclude = ['SA','SB','SC','SD','SE','SF']
colz = [i for i in range(132) if i not in cols_to_skip]

melt_IDvars = ['OkpasOk_Columns','GCRM #_Columns','Project & Opportunities_Columns','SITCOM or BACK-LOG or Investment_Columns',
                'Country of Destination_Columns','Contract currency (ISO3)_Columns','Year (T0)_Columns','Lead BL_Columns','Lead Country_Columns',
                'Lead Procurement Region_Columns','GBU_Columns','CBB (Contract Currency)_Columns','CBB Converted (EUR)_Columns',
                'P3 (%)_Columns','Total weighted CBB (EUR)_Columns','PSS available (Y/N)_Columns','Procurement scope_Columns',
                'Model_Columns','Model #_Columns','Procurement % of CBB (Model) (/100)_Columns','Total Procurement Volume (EUR)_Columns',
                '2020 (Actuals)_Columns','2021_Columns','2022_Columns','2023_Columns']

Seg_table = {'Seg': ['A', 'A', 'A', 'A', 'A',
                     'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
                     'C', 'C', 'C', 'C', 'C', 'C', 'C',
                     'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D',
                     'E', 'E', 'E', 'E', 'E',
                     'F', 'F', 'F', 'F'
                     ],
             'Cat': ['C12', 'C20', 'C21', 'C22', 'C23',
                     'C30', 'C31', 'C32', 'C34', 'C36', 'C37', 'C38', 'C39', 'C3A', 'C80', 'C81',
                     'C35', 'C40', 'C41', 'C42', 'C45-B', 'C46-A', 'C47',
                     'C43', 'C44', 'C45-A', 'C46-B', 'C46-C', 'C50', 'C51', 'C90', 'C91', 'C92', 'C93', 'C94',
                     'C60', 'C61', 'C62', 'C73', 'C74',
                     'C10', 'C13', 'C14', 'C15'
                     ]
             }

Seg_Cat_table = pd.DataFrame(data=Seg_table)


dt = pd.read_excel(data_file,
                   sheet_name=sheet,
                   header=[0,1],
                   keep_default_na=True,
                   #usecols=cols,
                   #skiprows=0,
                   index=True,
                   index_col=None
                   #index_col=[0,1]
                   )

dataset_cleaned = dt.fillna(0)
dataset_cleaned.rename({'Items':'Columns'}, level = 0, axis='columns', inplace=True)

dataset_cleaned1 = dataset_cleaned.drop(['2020 (Actuals)','2021','2022','2023'], axis=1, level=0)
dataset_cleaned2 = dataset_cleaned1.drop(['2020 (Actuals)','2021','2022','2023'], axis=1, level=1)

dataset_cleaned2.columns = dataset_cleaned2.columns.map('{0[1]}_{0[0]}'.format)

dataset_mlt = pd.melt(dataset_cleaned2, id_vars=['OkpasOk_Columns','GCRM #_Columns','Project & Opportunities_Columns','SITCOM or BACK-LOG or Investment_Columns',
                'Country of Destination_Columns','Contract currency (ISO3)_Columns','Year (T0)_Columns','Lead BL_Columns','Lead Country_Columns',
                'Lead Procurement Region_Columns','GBU_Columns','CBB (Contract Currency)_Columns','CBB Converted (EUR)_Columns',
                'P3 (%)_Columns','Total weighted CBB (EUR)_Columns','PSS available (Y/N)_Columns','Procurement scope_Columns',
                'Model_Columns','Model #_Columns','Procurement % of CBB (Model) (/100)_Columns','Total Procurement Volume (EUR)_Columns'
                ])

dataset_mlt['Cat']=dataset_mlt['variable'].apply(lambda x: x.split("_", 1)[0])
dataset_mlt['annee']=dataset_mlt['variable'].apply(lambda x: x.split("_", 1)[1])
dataset_mlt['Cat'] = dataset_mlt['Cat'].str.upper()
dataset_mlt['Cat'] = dataset_mlt['Cat'].str.strip()


dataset_mlt = dataset_mlt[['OkpasOk_Columns','GCRM #_Columns','Project & Opportunities_Columns','SITCOM or BACK-LOG or Investment_Columns',
'Country of Destination_Columns','Contract currency (ISO3)_Columns','Year (T0)_Columns','Lead BL_Columns','Lead Country_Columns',
'Lead Procurement Region_Columns','GBU_Columns','CBB (Contract Currency)_Columns','CBB Converted (EUR)_Columns',
'P3 (%)_Columns','Total weighted CBB (EUR)_Columns','PSS available (Y/N)_Columns','Procurement scope_Columns',
'Model_Columns','Model #_Columns','Procurement % of CBB (Model) (/100)_Columns','Total Procurement Volume (EUR)_Columns',
'variable','value','Cat','annee']].merge(Seg_Cat_table[['Seg', 'Cat']], on='Cat', how='left')

#display(dataset_mlt)
#print(dataset_cleaned.to_html())
#dataset_cleaned
dataset_mlt.to_csv('C:\\Users\\T0230575\\Desktop\\output.csv', sep=',', index = False)

